<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_010_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Exercise 0
import pandas as pd
from rdkit import Chem
df=pd.read_csv('steroid_dataset - my_data.csv')

def smilestomol(smile):
    mol=Chem.MolFromSmiles(smile)
    return mol
df['molecules']=df['Steroid_SMILES_grouped'].apply(smilestomol)
display(df)


assert df['molecules'].dtype=='object'
assert df['molecules'].count()==df['Entry'].count()

,Entry,Entry_Name,Protein_names,Gene_Names,Organism,Length,Sequence,Annotation_score,Rhea ID,ChEBI ID,source,Rhea_numeric_list,Steroid_Rhea_numeric,Steroid_ChEBI_grouped,Steroid_SMILES_grouped,mol_objects,molecules
0,A0A016VA76,A0A016VA76_9BILA,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,Acey_s0014.g2217 Acey-daf-22 Y032_0014g2217,Ancylostoma ceylanicum,531,MTKPKVFVIGVGMTKFCKPGSRDWDYPDMVKEAVTTALDDCKLKYS...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7820598ad1c0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b67ca0>
1,A0A023EUU2,A0A023EUU2_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7820593dccf0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b67d10>
2,A0A023EUU7,A0A023EUU7_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7820593dcc80>,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b67d80>
3,A0A023FCJ4,A0A023FCJ4_TRIIF,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Triatoma infestans (Assassin bug),540,RVKVYVVGVGMTKFYKPGKSDKDYPELAKEAIMKALEDARINHDDV...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7820593dc9e0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b67df0>
4,A0A026WVM4,A0A026WVM4_OOCBI,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,X777_13892,Ooceraea biroi (Clonal raider ant) (Cerapachys...,541,MVYKPKVYVIGVGMTKFEKPGRRDDFDYPQMAKEAVSKALQDSRIY...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7820593dc6d0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b67e60>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,W5MGD7,W5MGD7_LEPOC,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,SCP2,Lepisosteus oculatus (Spotted gar),563,TSHLHSQRHRTSYALCCYSQQQQMAPQKRRVFVVGVGMSKFEKPGA...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x78205941e960>,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aeca110>
1969,W5N409,W5N409_LEPOC,Hedgehog protein,NaN,Lepisosteus oculatus (Spotted gar),421,MDEMLLLTRIVLVGLTCLAFVSSGLACGPGRGYGRRRHPKKLTPLA...,5,59504,CHEBI:16113; CHEBI:143135,ChEBI,['59504'],59504,[CHEBI:16113],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x78205941e9d0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aeca180>
1970,W5P7N1,W5P7N1_SHEEP,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,LOC101117285,Ovis aries (Sheep),653,MEPFSRAERDGASNQPVNSRLELCPLESPSTPLPTQSSPVAPRMRT...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x78205941ea40>,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aeca1f0>
1971,W5P7N5,W5P7N5_SHEEP,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,LOC101117285,Ovis aries (Sheep),543,MSLAASQSLLRNRVFVVGVGMTKFTKPGVETRDYPDLAKEAGQKAL...,5,62960,CHEBI:17759,ChEBI,['62960'],62960,[CHEBI:17759],C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x78205941eab0>,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aeca260>


In [20]:
#Exercise 1
from rdkit.Chem import AllChem, MACCSkeys

def fingerprint(mol):
  morgan_fps = {}
  bit_infos = {}

  for radius in [1, 2, 3]:
      bit_info = {}
      fp = AllChem.GetMorganFingerprintAsBitVect(
          mol,
          radius=radius,
          nBits=1024,
          bitInfo=bit_info
      )
      morgan_fps[radius] = fp
      bit_infos[radius] = bit_info

  return morgan_fps

def mackeyfingerprint(mol):
  maccs_fp = MACCSkeys.GenMACCSKeys(mol)
  return maccs_fp

def rdkittopological(mol):
  rdk_fp = Chem.RDKFingerprint(mol)
  return rdk_fp

mol=df.at[0,'molecules']
print(type(fingerprint(mol)[2]))
print(type(mackeyfingerprint(mol)))
print(type(rdkittopological(mol)))

print(len(fingerprint(mol)[1]))
print(len(fingerprint(mol)[2]))
print(len(fingerprint(mol)[3]))

print(len(mackeyfingerprint(mol)))
print(len(rdkittopological(mol)))

<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
1024
1024
1024
167
2048


[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator
[07:58:46] DEPRECATION WARNING: please use MorganGenerator


In [ ]:
#Exercise 2
from rdkit import DataStructs



